In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from operator import attrgetter
import matplotlib.colors as mcolors
import csv
import numpy as np

In [2]:
df = pd.read_csv("JiwaPlusTrans.csv", sep = ";",thousands = ".")
df

,id_transaction,id_user,name,phone,transaction_date,Month,gross_sales,shipment_cost,total_discount,total_net_sales,trasaction_type
0,6635,6056,luluk ari,85333690115,2020-09-16,2020-09,18000,0,18000,0,Pickup Order
1,7216,15356,gilar rakasiwi,89669025942,2020-09-16,2020-09,18000,0,18000,0,Pickup Order
2,7639,30738,dimasfa91@gmail.com,89688145012,2020-09-16,2020-09,18000,0,18000,0,Pickup Order
3,7687,39492,jauharul firdaus,83866954256,2020-09-16,2020-09,18000,0,18000,0,Pickup Order
4,1958,9622,dhani setiaji,85156616854,2020-09-16,2020-09,23000,0,18000,5000,Pickup Order
...,...,...,...,...,...,...,...,...,...,...,...
104947,175536,275463,Rachel,82153791681,2021-09-29,2021-09,15000,0,0,15000,Pickup Order
104948,175552,274855,Mei Ramandey,87785107033,2021-09-29,2021-09,23000,0,0,23000,Pickup Order
104949,175788,264845,Putri,89655459263,2021-09-30,2021-09,20000,0,0,20000,Pickup Order
104950,175840,268457,Randy Gunawan,87781995971,2021-09-30,2021-09,25000,0,0,25000,Pickup Order


In [3]:
months = df["Month"].unique()
print(months)

['2020-09' '2020-10' '2020-11' '2020-12' '2021-01' '2021-02' '2021-03'
 '2021-04' '2021-05' '2021-06' '2021-07' '2021-08' '2021-09']


In [4]:
cohorts = {}
userset = set()

In [5]:
for x in months:
    dfx = df[df['Month'] == x]
    listx = set(dfx['id_user']) - userset
    userset.update(listx)
    cohorts[x] = listx


In [6]:
for m in months:
    print(len(cohorts[m]))

31806
2627
1295
2033
1056
1652
1473
2031
1749
1504
2862
1761
2326


In [7]:
cohorts2={}
for d in cohorts :
    cohorts2[d] = list(cohorts[d])

In [8]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
id_transaction,104952.0,8.823499e+04,5.199896e+04,5.0,3.995150e+04,8.809850e+04,1.349545e+05,1.760890e+05
id_user,104952.0,1.397615e+05,8.732339e+04,194.0,5.454800e+04,1.651270e+05,2.091632e+05,2.812440e+05
phone,104952.0,9.537772e+10,1.306917e+11,811100727.0,8.128813e+10,8.224934e+10,8.580293e+10,8.989983e+11
gross_sales,104952.0,4.207257e+04,4.776437e+04,5000.0,1.800000e+04,2.300000e+04,5.100000e+04,2.650000e+06
shipment_cost,104952.0,4.348054e+02,2.685624e+03,0.0,0.000000e+00,0.000000e+00,0.000000e+00,7.400000e+04
total_discount,104952.0,1.300303e+04,1.612927e+04,0.0,0.000000e+00,1.500000e+04,1.800000e+04,1.650000e+06
total_net_sales,104952.0,2.950435e+04,4.152076e+04,0.0,0.000000e+00,2.000000e+04,4.000000e+04,2.450000e+06


In [ ]:
filename = "Users.csv"
with open(filename, 'a') as csvfile:
    wr = csv.writer(csvfile, delimiter='+')
    for key, value in cohorts2.items():
        wr.writerow([key, *value])

In [11]:
n_orders = df.groupby(['id_user'])['id_transaction'].nunique()
mult_orders_perc = np.sum(n_orders > 1) / df['id_user'].nunique()
print(f'{100 * mult_orders_perc:.2f}% of customers ordered more than once.')

20.04% of customers ordered more than once.


In [18]:
df['cohort'] = ""
df.head()

,id_transaction,id_user,name,phone,transaction_date,Month,gross_sales,shipment_cost,total_discount,total_net_sales,trasaction_type,cohort
0,6635,6056,luluk ari,85333690115,2020-09-16,2020-09,18000,0,18000,0,Pickup Order,
1,7216,15356,gilar rakasiwi,89669025942,2020-09-16,2020-09,18000,0,18000,0,Pickup Order,
2,7639,30738,dimasfa91@gmail.com,89688145012,2020-09-16,2020-09,18000,0,18000,0,Pickup Order,
3,7687,39492,jauharul firdaus,83866954256,2020-09-16,2020-09,18000,0,18000,0,Pickup Order,
4,1958,9622,dhani setiaji,85156616854,2020-09-16,2020-09,23000,0,18000,5000,Pickup Order,


In [34]:
for key, items in cohorts2.items():
    for item in items:
        df.loc[df['id_user'] == item, 'cohort'] = key

In [35]:
df.head()

,id_transaction,id_user,name,phone,transaction_date,Month,gross_sales,shipment_cost,total_discount,total_net_sales,trasaction_type,cohort
0,6635,6056,luluk ari,85333690115,2020-09-16,2020-09,18000,0,18000,0,Pickup Order,2020-09
1,7216,15356,gilar rakasiwi,89669025942,2020-09-16,2020-09,18000,0,18000,0,Pickup Order,2020-09
2,7639,30738,dimasfa91@gmail.com,89688145012,2020-09-16,2020-09,18000,0,18000,0,Pickup Order,2020-09
3,7687,39492,jauharul firdaus,83866954256,2020-09-16,2020-09,18000,0,18000,0,Pickup Order,2020-09
4,1958,9622,dhani setiaji,85156616854,2020-09-16,2020-09,23000,0,18000,5000,Pickup Order,2020-09


In [37]:
filename = 'cohortjiwaplus.csv'
df.to_csv(filename, sep=';')